<div class="jumbotron">
  <h1 class="display-3">First (basic) lesson with Abinit and AbiPy</h1>
  <p class="lead">The H<sub>2</sub> molecule</p> 
  <hr class="my-4">
  <p>This lesson aims at showing how to get the following physical properties:
      
      the (pseudo) total energy
      the bond length
      the charge density
      the atomisation energy
   
  </p>
  <p>You will learn about the two input files, the basic input variables, the existence of defaults, the actions of the parser, and the use of the multi-dataset feature. You will also learn about the two output files as well as the density file. This first lesson covers the sections 1, 3, 4 and 6 of the abinit help file.</p>
<p>
The very first step is a detailed tour of the input and output files: you are like a tourist, and you discover a town in a coach. You will have a bit more freedom after that first step. It is supposed that you have some good knowledge of UNIX/Linux.</p>
</p>
  <p class="lead">
    <a class="btn btn-primary btn-lg" href="#" role="button">Learn more</a>
  </p>
</div>

In [1]:
# Use this at the beginning of your script so that your code will be compatible with python3
from __future__ import print_function, division, unicode_literals

import numpy as np

from abipy import abilab
%matplotlib notebook

In [2]:
from mkflow import gs_input
abilab.print_source(gs_input)

In [3]:
inp = gs_input()

In [4]:
print("ecut", inp["ecut"])

ecut 10


In [5]:
inp

<AbinitInput at 4821470792>

In [6]:
print(inp.structure)
print("Volume", inp.structure.volume)

Full Formula (H2)
Reduced Formula: H2
abc   :   5.291772   5.291772   5.291772
angles:  90.000000  90.000000  90.000000
Sites (2)
  #  SP        a    b    c
---  ----  -----  ---  ---
  0  H     -0.07    0    0
  1  H      0.07    0    0
Volume 148.184711276


In [7]:
print(inp.pseudos[0])

<NcAbinitPseudo: 01h.pspgth>
  summary: Goedecker-Teter-Hutter  Wed May  8 14:27:44 EDT 1996
  number of valence electrons: 1.0
  maximum angular momentum: s
  angular momentum for local part: s
  XC correlation: LDA_XC_TETER93
  supports spin-orbit: False
  radius for non-linear core correction: 0.0
  hint for low accuracy: ecut: 0.0, pawecutdg: 0.0
  hint for normal accuracy: ecut: 0.0, pawecutdg: 0.0
  hint for high accuracy: ecut: 0.0, pawecutdg: 0.0


<div class="alert alert-danger">
Blah blah blah
</div>

## Computation of the interatomic distance (method 1)

In [8]:
abo = abilab.abiopen("flow_h2/w0/t0/run.abo")
print(abo)

ndtset: 1, completed: True
Full Formula (H2)
Reduced Formula: H2
abc   :   5.291772   5.291772   5.291772
angles:  90.000000  90.000000  90.000000

Spglib space group info (magnetic symmetries are not taken into account).
Spacegroup: P4/mmm (123), Hall: -P 4 2, Abinit spg_number: 123
Crystal_system: tetragonal, Lattice_type: tetragonal, Point_group: 4/mmm

  Idx  Symbol    Reduced_Coords              Wyck      EqIdx
-----  --------  --------------------------  ------  -------
    0  H         -0.05000 +0.00000 +0.00000  g             0
    1  H         +0.05000 +0.00000 +0.00000  g             0

Abinit Spacegroup: spgid: 123, num_spatial_symmetries: 16, has_timerev: True, symmorphic: False


In [9]:
print(abo.events)

Events found in /Users/gmatteo/git_repos/abitutorials/abitutorials/base1/flow_h2/w0/t0/run.abo

num_errors: 0, num_warnings: 0, num_comments: 0, completed: False



In [10]:
abo.plot()

<IPython.core.display.Javascript object>

exception while parsing file /Users/gmatteo/git_repos/abitutorials/abitutorials/base1/flow_h2/w0/t0/run.abo:
/Users/gmatteo/git_repos/abitutorials/abitutorials/base1/flow_h2/w0/t0/run.abo: No timer section found


In [11]:
with abilab.abiopen("flow_h2/w0/t0/outdata/out_GSR.nc") as gsr:
    print(gsr)

================================= File Info =================================
Name: out_GSR.nc
Directory: /Users/gmatteo/git_repos/abitutorials/abitutorials/base1/flow_h2/w0/t0/outdata
Size: 8.20 kb
Access Time: Tue Oct 10 21:32:48 2017
Modification Time: Tue Oct 10 21:27:35 2017
Change Time: Tue Oct 10 21:27:35 2017

================================= Structure =================================
Full Formula (H2)
Reduced Formula: H2
abc   :   5.291772   5.291772   5.291772
angles:  90.000000  90.000000  90.000000
Sites (2)
  #  SP        a    b    c  cartesian_forces
---  ----  -----  ---  ---  --------------------------------------------------
  0  H     -0.05    0    0  [-19.54779666  -0.          -0.        ] eV ang^-1
  1  H      0.05    0    0  [ 19.54779666  -0.          -0.        ] eV ang^-1

Stress tensor (Cartesian coordinates in Ha/Bohr**3):
[[ -3.65644463e-04   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   5.46899250e-05   0.00000000e+00]
 [  0.00000000e+00   0.00000

In [12]:
abilab.abiopen

<function abipy.abilab.abiopen>

In [13]:
def hh_dist(gsr):
    """This function receives a GSR file and computes the H-H distance"""
    cart_coords = gsr.structure.cart_coords
    l = np.linalg.norm(cart_coords[1] - cart_coords[0])
    return "hh_dist", l

with abilab.GsrRobot.from_dir("flow_h") as robot:
    table = robot.get_dataframe(funcs=hh_dist)
    table = table.sort_values(by="hh_dist")  

ValueError: flow_h: no such directory

In [ ]:
table.keys()

In [ ]:
table[["hh_dist", "energy"]]

In [ ]:
table.plot(x="hh_dist", y="energy", style="-o")

In [ ]:
table.plot(x="hh_dist", y="max_force", style="-o")

## Computation of the charge density

In [ ]:
with abilab.abiopen("flow_h2/w0/t10/outdata/out_DEN.nc") as denfile:
    print(denfile)
    density = denfile.density

In [ ]:
fig = density.plot_line(0, 1)